In [1]:
import numpy as np

import time
import mujoco
import mujoco.viewer

import mujoco
import numpy as np
import matplotlib.pyplot as plt
import mujoco.viewer

import mediapy as media
import time

import matplotlib.pyplot as plt
from scipy.linalg import null_space
import PyKDL as kdl

In [2]:
from main_21aug import LeapNodeMujoco
from leaphand_kinematics import LeapHandKinematics
from grasp import GraspClass

model_path = '/home/sysidea/leap_hand_mujoco/model/leap hand/leaphand_19.8.24.xml'
leap_hand = LeapNodeMujoco(model_path)
urdf_path = '/home/sysidea/leap_hand_mujoco/model/leap hand/robot.urdf'
LHK = LeapHandKinematics(urdf_path)


m = mujoco.MjModel.from_xml_path(model_path)
d = mujoco.MjData(m)

Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='0']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='1']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='2']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='3']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='4']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='5']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='6']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='7']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='8']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='9']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='10']
Unknown tag "joint_properties" in /robot[@name='Right LEAP Hand']/joint[@name='11']
Un

In [5]:
framerate=30

#go near the contact point
while leap_hand.d.time < 2:

    leap_hand.apply_controls_palm(0)
    leap_hand.apply_controls_wrist(0)

    #placeholder code 
    #index finger
    # [a,b,c,d]=LHK.perform_ik('fingertip',kdl.Frame(kdl.Rotation.RPY(0, 0, 0),
    #                         kdl.Vector(0, 0, 0)),)
    # #thumb
    # [e,f,g,h]=LHK.perform_ik('thumb_fingertip',kdl.Frame(kdl.Rotation.RPY(0, 0, 0),
    #                         kdl.Vector(0, 0, 0)),)
    
    # leap_hand.apply_controls([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    leap_hand.apply_controls([0.88168, 0, 0.438445, 0.236, 0, 0, 0, 0, 0, 0, 0, 0, 1.54432, 0, -0.7815, 0.7852])

    leap_hand.step_video(framerate,'closeup')
       
#apply force to hold object
while leap_hand.d.time < 4 and leap_hand.d.time>2:
    
    leap_hand.apply_controls_palm(0)
    leap_hand.apply_controls_wrist(0)
    
    #placeholder code
    #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)

    leap_hand.step_video(framerate,'closeup')

#lift object
while leap_hand.d.time > 4 and leap_hand.d.time<6:
    
    leap_hand.apply_controls_palm(0.2)
    leap_hand.apply_controls_wrist(0)

    #placeholder
    #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)
    
    leap_hand.step_video(framerate,'closeup')

#rotate palm
while leap_hand.d.time > 6 and leap_hand.d.time<8:

    leap_hand.apply_controls_wrist(3.14)
    
    #placeholder
    #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)
    
    leap_hand.step_video(framerate,'closeup')

#bring object down to palm
while leap_hand.d.time > 8 and leap_hand.d.time<10:
    
    leap_hand.apply_controls_palm(1)
    leap_hand.apply_controls_wrist(3.14)

    #placeholder
    #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)
    
    leap_hand.step_video(framerate,'closeup')

# #drop object
# while leap_hand.d.time > 8 and leap_hand.d.time<10:
    
#     leap_hand.apply_controls_palm(1)
#     leap_hand.apply_controls_wrist(3.14)

#     #leap_hand.apply_controls(np.zeros(16))
    
#     leap_hand.step_video(framerate)

# #use thumb to push object
# while leap_hand.d.time > 8 and leap_hand.d.time<10:
    
#     leap_hand.apply_controls_palm(1)
#     leap_hand.apply_controls_wrist(3.14)

#      #leap_hand.torque_control(null_space(I-pinv(G)*G)+pinv(G)*(Kp*(X-Xd)))
    
#     leap_hand.step_video(framerate)

# #middle and last finger flat end on cube
# while leap_hand.d.time > 10 and leap_hand.d.time<12:
    
#     leap_hand.apply_controls_palm(1)
#     leap_hand.apply_controls_wrist(3.14)

#      #leap_hand.inv_kin(mid,other)
    
#     leap_hand.step_video(framerate)

# #apply force
# while leap_hand.d.time > 10 and leap_hand.d.time<12:
    
#     leap_hand.apply_controls_palm(1)
#     leap_hand.apply_controls_wrist(3.14)

#      #Tau=J.T*F
    
#     leap_hand.step_video(framerate)
        
    
leap_hand.play_video(framerate)
    

Total frames captured: 301


This browser does not support the video tag.


Grasp Matrix and Jacobian (Index and Thumb)

In [ ]:
# Define matrices
n=2
rpalm_inv=np.linalg.inv(np.reshape(leap_hand.d.xmat[3],(3,3)))
r_cube=np.reshape(leap_hand.d.xmat[0],(3,3))
r_theta=rpalm_inv*r_cube
contact_orientations=[np.reshape(leap_hand.d.site_xmat[0],(3,3)),np.reshape(leap_hand.d.site_xmat[0],(3,3))]
bs=[np.reshape([0, 0.027,0],(3,1)),np.reshape([0, -0.027,0],(3,1))]

Grasp=GraspClass()
G_index_thumb = Grasp.G(n,contact_orientations,r_theta,bs)
# Jh_index_thumb = np.zeros([6, 3])
# I = np.eye(G.shape[1])

# # Compute the pseudoinverse of G
# pinv_G = np.linalg.pinv(G)

# # Compute the projection matrix (I - pinv_G @ G)
# projection_matrix = I - pinv_G @ G

# # # Compute the null space of the projection matrix
# print(null_space(projection_matrix))
print(G_index_thumb)


<key qpos='-0.02 0.005 0 1 0 0 0 0 0 0.88168 0 0.438445 0.236 0 0 0 0 0 0 0 0 1.54432 0 -0.7815 0.7852'/>
[0.88168, 0, 0.438445, 0.236, 0, 0, 0, 0, 0, 0, 0, 0, 1.54432, 0, -0.7815, 0.7852]